# OBS: Ajustar o caminho para os arquivos.csv e a senha do postgres

# Database creation and data loading script for MC536 Project.

This script connects to a PostgreSQL database, creates the necessary tables
based on a predefined schema, and populates them with data from three
CSV files:
1. suite_of_food_security_indicators.csv
2. production_crops_livestock.csv
3. agricultural_census.csv

# 1. Requirements

In [ ]:
import psycopg2
import csv
import sys # Used for flushing output in Jupyter
import pandas as pd

# --- Database Connection Parameters ---
# !! IMPORTANT: Replace "pswd" with your actual PostgreSQL password !!
DB_NAME = "mc536_project"
DB_USER = "postgres"
DB_PASS = "pwd" # <--- CHANGE THIS TO YOUR PASSWORD
DB_HOST = "localhost"
DB_PORT = "5432"

# --- CSV File Paths ---
# !! IMPORTANT: Update these paths to where your CSV files are located !!
FOOD_SECURITY_CSV = 'suite_of_food_security_indicators.csv'
PRODUCTION_CSV = 'production_crops_livestock.csv'
CENSUS_CSV = 'agricultural_census.csv'

# 2. Global variables and functions

In [4]:
# These dictionaries store mappings from unique identifiers (like names or codes)
# to the auto-generated primary keys (IDs) in the database. This prevents
# duplicate entries in lookup tables (Country, Indicator, Product, Farm_Size_Range)
# and ensures correct foreign key references.

# { (country_name, iso_code) : country_id }
country_map = {}
country_id_counter = 1

# { (indicator_name, unit) : indicator_id }
food_indicator_map = {}
food_indicator_id_counter = 1

# { (product_name, product_type) : product_id }
agri_product_map = {}
agri_product_id_counter = 1

# { (lower_limit, upper_limit) : range_id }
farm_size_range_map = {}
farm_size_range_id_counter = 1

# { (country_id, census_year) : census_id } - To handle unique censuses
agri_census_map = {}
agri_census_id_counter = 1

# Counters for fact tables (where IDs are generated sequentially for each row)
food_measurement_id_counter = 1
agri_production_id_counter = 1
rural_distribution_id_counter = 1

# --- Helper Function for Safe Data Conversion ---

def safe_int(value, default=None):
    """
    Safely converts a value to an integer.
    Returns default if conversion fails or input is empty/None.
    """
    if value in (None, ''):
        return default
    try:
        return int(value)
    except (ValueError, TypeError):
        return default

def safe_float(value, default=None):
    """
    Safely converts a value to a float.
    Returns default if conversion fails or input is empty/None.
    """
    if value in (None, ''):
        return default
    try:
        # Handle potential comma as decimal separator if needed, assuming '.' for now
        return float(str(value).replace(',', '.'))
    except (ValueError, TypeError):
        return default

# 3. Database Schema Creation Function

In [4]:
def create_tables(cursor):
    """
    Creates the database tables based on the predefined SQL schema.

    Args:
        cursor: A psycopg2 cursor object connected to the database.
    """
    print("Creating tables...")
    sql_schema = """
    BEGIN;

    -- Drop existing tables if they exist (optional, for clean runs)
    -- Use with caution, this will delete all data!
    DROP TABLE IF EXISTS public."Food_Security_Measurement" CASCADE;
    DROP TABLE IF EXISTS public."Agricultural_Production" CASCADE;
    DROP TABLE IF EXISTS public."Rural_Property_Distributuon" CASCADE; -- Typo matches schema
    DROP TABLE IF EXISTS public."Agricultural_Census" CASCADE;
    DROP TABLE IF EXISTS public."Food_Security_Indicator" CASCADE;
    DROP TABLE IF EXISTS public."Agricultural_Product" CASCADE;
    DROP TABLE IF EXISTS public."Farm_Size_Range" CASCADE;
    DROP TABLE IF EXISTS public."Country" CASCADE;


    CREATE TABLE IF NOT EXISTS public."Country"
    (
        "ID" integer NOT NULL,
        name character varying NOT NULL,
        iso_code character varying NOT NULL,
        PRIMARY KEY ("ID"),
        UNIQUE (name, iso_code) -- Ensure country uniqueness
    );

    CREATE TABLE IF NOT EXISTS public."Farm_Size_Range"
    (
        "ID" integer NOT NULL,
        lower_limit integer NOT NULL,
        upper_limit integer NOT NULL,
        PRIMARY KEY ("ID"),
        UNIQUE (lower_limit, upper_limit) -- Ensure range uniqueness
    );

    CREATE TABLE IF NOT EXISTS public."Agricultural_Product"
    (
        "ID" integer NOT NULL,
        name character varying NOT NULL,
        type character varying NOT NULL,
        PRIMARY KEY ("ID"),
        UNIQUE (name, type) -- Ensure product uniqueness
    );

    CREATE TABLE IF NOT EXISTS public."Food_Security_Indicator"
    (
        "ID" integer NOT NULL,
        name character varying NOT NULL,
        unit character varying NOT NULL,
        PRIMARY KEY ("ID"),
        UNIQUE (name, unit) -- Ensure indicator uniqueness
    );

    CREATE TABLE IF NOT EXISTS public."Agricultural_Census"
    (
        "ID" integer NOT NULL,
        census_year integer NOT NULL,
        decade character varying NOT NULL,
        "Country_ID" integer NOT NULL,
        PRIMARY KEY ("ID"),
        UNIQUE ("Country_ID", census_year) -- Ensure one census per country/year
    );

    -- Typo "Distributuon" kept to match provided schema
    CREATE TABLE IF NOT EXISTS public."Rural_Property_Distributuon"
    (
        "ID" integer NOT NULL,
        total_area_ha real NOT NULL,
        number_of_properties integer NOT NULL,
        "Size_Range_ID" integer NOT NULL,
        "Agricultural_Census_ID" integer NOT NULL,
        PRIMARY KEY ("ID")
    );

    CREATE TABLE IF NOT EXISTS public."Agricultural_Production"
    (
        "ID" integer NOT NULL,
        year integer NOT NULL,
        production_tons real NOT NULL,
        "Product_ID" integer NOT NULL,
        "Country_ID" integer NOT NULL,
        number_of_animals integer, -- Nullable
        harvested_area_ha real,    -- Nullable
        PRIMARY KEY ("ID")
    );

    CREATE TABLE IF NOT EXISTS public."Food_Security_Measurement"
    (
        "ID" integer NOT NULL,
        year integer NOT NULL,
        value real NOT NULL,
        "Country_ID" integer NOT NULL,
        "Indicator_ID" integer NOT NULL,
        PRIMARY KEY ("ID")
    );

    -- Add Foreign Key Constraints

    ALTER TABLE IF EXISTS public."Agricultural_Census" DROP CONSTRAINT IF EXISTS fk_census_country;
    ALTER TABLE IF EXISTS public."Agricultural_Census"
        ADD CONSTRAINT fk_census_country FOREIGN KEY ("Country_ID")
        REFERENCES public."Country" ("ID") MATCH SIMPLE
        ON UPDATE NO ACTION
        ON DELETE CASCADE; -- Cascade delete if country is removed

    ALTER TABLE IF EXISTS public."Rural_Property_Distributuon" DROP CONSTRAINT IF EXISTS fk_dist_census;
    ALTER TABLE IF EXISTS public."Rural_Property_Distributuon"
        ADD CONSTRAINT fk_dist_census FOREIGN KEY ("Agricultural_Census_ID")
        REFERENCES public."Agricultural_Census" ("ID") MATCH SIMPLE
        ON UPDATE NO ACTION
        ON DELETE CASCADE; -- Cascade delete if census is removed

    ALTER TABLE IF EXISTS public."Rural_Property_Distributuon" DROP CONSTRAINT IF EXISTS fk_dist_size_range;
    ALTER TABLE IF EXISTS public."Rural_Property_Distributuon"
        ADD CONSTRAINT fk_dist_size_range FOREIGN KEY ("Size_Range_ID")
        REFERENCES public."Farm_Size_Range" ("ID") MATCH SIMPLE
        ON UPDATE NO ACTION
        ON DELETE CASCADE; -- Consider if range deletion should cascade

    ALTER TABLE IF EXISTS public."Agricultural_Production" DROP CONSTRAINT IF EXISTS fk_prod_country;
    ALTER TABLE IF EXISTS public."Agricultural_Production"
        ADD CONSTRAINT fk_prod_country FOREIGN KEY ("Country_ID")
        REFERENCES public."Country" ("ID") MATCH SIMPLE
        ON UPDATE NO ACTION
        ON DELETE CASCADE;

    ALTER TABLE IF EXISTS public."Agricultural_Production" DROP CONSTRAINT IF EXISTS fk_prod_product;
    ALTER TABLE IF EXISTS public."Agricultural_Production"
        ADD CONSTRAINT fk_prod_product FOREIGN KEY ("Product_ID")
        REFERENCES public."Agricultural_Product" ("ID") MATCH SIMPLE
        ON UPDATE NO ACTION
        ON DELETE CASCADE;

    ALTER TABLE IF EXISTS public."Food_Security_Measurement" DROP CONSTRAINT IF EXISTS fk_meas_country;
    ALTER TABLE IF EXISTS public."Food_Security_Measurement"
        ADD CONSTRAINT fk_meas_country FOREIGN KEY ("Country_ID")
        REFERENCES public."Country" ("ID") MATCH SIMPLE
        ON UPDATE NO ACTION
        ON DELETE CASCADE;

    ALTER TABLE IF EXISTS public."Food_Security_Measurement" DROP CONSTRAINT IF EXISTS fk_meas_indicator;
    ALTER TABLE IF EXISTS public."Food_Security_Measurement"
        ADD CONSTRAINT fk_meas_indicator FOREIGN KEY ("Indicator_ID")
        REFERENCES public."Food_Security_Indicator" ("ID") MATCH SIMPLE
        ON UPDATE NO ACTION
        ON DELETE CASCADE;

    COMMIT; -- Commit transaction for schema creation
    """
    try:
        cursor.execute(sql_schema)
        print("Tables created successfully (or already exist).")
    except psycopg2.Error as e:
        print(f"Error creating tables: {e}")
        # No rollback needed here as BEGIN/COMMIT is in the SQL string
        raise # Re-raise the exception to stop the script if table creation fails

# 4. Data Loading Functions

In [5]:
def load_food_security_data(cursor, file_path):
    """
    Loads data from the food security CSV into the relevant tables.
    Updates Country, Food_Security_Indicator, and inserts into
    Food_Security_Measurement.

    Args:
        cursor: A psycopg2 cursor object.
        file_path (str): The path to the suite_of_food_security_indicators.csv file.
    """
    print(f"\nLoading data from {file_path}...")
    global country_map, country_id_counter
    global food_indicator_map, food_indicator_id_counter
    global food_measurement_id_counter

    processed_rows = 0
    skipped_rows = 0
    try:
        with open(file_path, 'r', encoding='utf-8') as csvfile:
            reader = csv.reader(csvfile)
            header = next(reader) # Skip header row

            for row in reader:
                try:
                    # Extract data from CSV row
                    iso_code = row[0].strip()
                    country_name = row[1].strip()
                    fs_year = safe_int(row[2])
                    fs_value = safe_float(row[3])
                    fs_indicator_name = row[4].strip()
                    fs_indicator_unit = row[5].strip()

                    # --- Validate essential data ---
                    if not all([iso_code, country_name, fs_year is not None, fs_value is not None, fs_indicator_name, fs_indicator_unit]):
                        # print(f"Skipping row due to missing essential data: {row}")
                        skipped_rows += 1
                        continue

                    # --- Get or Create Country ID ---
                    country_key = (country_name, iso_code)
                    if country_key not in country_map:
                        current_country_id = country_id_counter
                        cursor.execute(
                            """
                            INSERT INTO public."Country" ("ID", name, iso_code)
                            VALUES (%s, %s, %s)
                            ON CONFLICT (name, iso_code) DO NOTHING;
                            """,
                            (current_country_id, country_name, iso_code)
                        )
                        # Fetch the ID whether inserted or already exists
                        cursor.execute(
                            'SELECT "ID" FROM public."Country" WHERE name = %s AND iso_code = %s',
                            (country_name, iso_code)
                        )
                        result = cursor.fetchone()
                        if result:
                           country_id = result[0]
                           country_map[country_key] = country_id
                           if country_id >= country_id_counter: # If ID came from existing row or new insert
                               country_id_counter = country_id + 1 # Ensure next ID is unique
                        else:
                             # This case should ideally not happen with ON CONFLICT, but handle defensively
                             print(f"Warning: Could not retrieve ID for country: {country_name}, {iso_code}")
                             skipped_rows += 1
                             continue

                    else:
                        country_id = country_map[country_key]

                    # --- Get or Create Food Security Indicator ID ---
                    indicator_key = (fs_indicator_name, fs_indicator_unit)
                    if indicator_key not in food_indicator_map:
                        current_indicator_id = food_indicator_id_counter
                        cursor.execute(
                            """
                            INSERT INTO public."Food_Security_Indicator" ("ID", name, unit)
                            VALUES (%s, %s, %s)
                            ON CONFLICT (name, unit) DO NOTHING;
                            """,
                            (current_indicator_id, fs_indicator_name, fs_indicator_unit)
                        )
                        # Fetch the ID
                        cursor.execute(
                            'SELECT "ID" FROM public."Food_Security_Indicator" WHERE name = %s AND unit = %s',
                            (fs_indicator_name, fs_indicator_unit)
                        )
                        result = cursor.fetchone()
                        if result:
                           indicator_id = result[0]
                           food_indicator_map[indicator_key] = indicator_id
                           if indicator_id >= food_indicator_id_counter:
                               food_indicator_id_counter = indicator_id + 1
                        else:
                             print(f"Warning: Could not retrieve ID for indicator: {fs_indicator_name}")
                             skipped_rows += 1
                             continue
                    else:
                        indicator_id = food_indicator_map[indicator_key]

                    # --- Insert Food Security Measurement ---
                    measurement_id = food_measurement_id_counter
                    cursor.execute(
                        """
                        INSERT INTO public."Food_Security_Measurement"
                        ("ID", year, value, "Country_ID", "Indicator_ID")
                        VALUES (%s, %s, %s, %s, %s);
                        """,
                        (measurement_id, fs_year, fs_value, country_id, indicator_id)
                    )
                    food_measurement_id_counter += 1
                    processed_rows += 1

                except (IndexError, ValueError, TypeError) as e:
                    print(f"Skipping row due to data error: {row} - Error: {e}")
                    skipped_rows += 1
                except psycopg2.Error as e:
                    print(f"Database error processing row: {row} - Error: {e}")
                    conn.rollback() # Rollback the specific failed insert if needed, or rely on main rollback
                    skipped_rows += 1

                # Print progress occasionally
                if (processed_rows + skipped_rows) % 1000 == 0:
                    print(f"  Processed {processed_rows + skipped_rows} rows ({processed_rows} inserted, {skipped_rows} skipped)...", end='\r')
                    sys.stdout.flush() # Force output in Jupyter

    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        raise
    finally:
        print(f"\nFinished loading {file_path}. Total rows processed: {processed_rows + skipped_rows}, Inserted: {processed_rows}, Skipped: {skipped_rows}.")


def load_production_data(cursor, file_path):
    """
    Loads data from the production CSV into relevant tables.
    Updates Country, Agricultural_Product, and inserts into
    Agricultural_Production.

    Args:
        cursor: A psycopg2 cursor object.
        file_path (str): The path to the production_crops_livestock.csv file.
    """
    print(f"\nLoading data from {file_path}...")
    global country_map, country_id_counter
    global agri_product_map, agri_product_id_counter
    global agri_production_id_counter

    processed_rows = 0
    skipped_rows = 0
    try:
        with open(file_path, 'r', encoding='utf-8') as csvfile:
            # Handle potential quotes in country names
            reader = csv.reader(csvfile, quotechar='"', skipinitialspace=True)
            header = next(reader) # Skip header row

            for row in reader:
                try:
                    # Extract data from CSV row
                    iso_code = row[0].strip()
                    country_name = row[1].strip()
                    prod_year = safe_int(row[2])
                    prod_tons = safe_float(row[3])
                    # Handle potentially empty numeric fields -> None
                    prod_n_animals = safe_int(row[4], default=None)
                    prod_harvested_area = safe_float(row[5], default=None)
                    agri_product_name = row[6].strip()
                    agri_product_type = row[7].strip()

                    # --- Validate essential data ---
                    if not all([iso_code, country_name, prod_year is not None, prod_tons is not None, agri_product_name, agri_product_type]):
                        # print(f"Skipping row due to missing essential data: {row}")
                        skipped_rows += 1
                        continue

                    # --- Get or Create Country ID ---
                    # (Reuses logic and map from previous function)
                    country_key = (country_name, iso_code)
                    if country_key not in country_map:
                        current_country_id = country_id_counter
                        cursor.execute(
                            """
                            INSERT INTO public."Country" ("ID", name, iso_code)
                            VALUES (%s, %s, %s)
                            ON CONFLICT (name, iso_code) DO NOTHING;
                            """,
                            (current_country_id, country_name, iso_code)
                        )
                        cursor.execute(
                            'SELECT "ID" FROM public."Country" WHERE name = %s AND iso_code = %s',
                            (country_name, iso_code)
                        )
                        result = cursor.fetchone()
                        if result:
                           country_id = result[0]
                           country_map[country_key] = country_id
                           if country_id >= country_id_counter:
                               country_id_counter = country_id + 1
                        else:
                             print(f"Warning: Could not retrieve ID for country: {country_name}, {iso_code}")
                             skipped_rows += 1
                             continue
                    else:
                        country_id = country_map[country_key]

                    # --- Get or Create Agricultural Product ID ---
                    product_key = (agri_product_name, agri_product_type)
                    if product_key not in agri_product_map:
                        current_product_id = agri_product_id_counter
                        cursor.execute(
                            """
                            INSERT INTO public."Agricultural_Product" ("ID", name, type)
                            VALUES (%s, %s, %s)
                            ON CONFLICT (name, type) DO NOTHING;
                            """,
                            (current_product_id, agri_product_name, agri_product_type)
                        )
                        cursor.execute(
                            'SELECT "ID" FROM public."Agricultural_Product" WHERE name = %s AND type = %s',
                            (agri_product_name, agri_product_type)
                        )
                        result = cursor.fetchone()
                        if result:
                           product_id = result[0]
                           agri_product_map[product_key] = product_id
                           if product_id >= agri_product_id_counter:
                               agri_product_id_counter = product_id + 1
                        else:
                            print(f"Warning: Could not retrieve ID for product: {agri_product_name}")
                            skipped_rows += 1
                            continue
                    else:
                        product_id = agri_product_map[product_key]

                    # --- Insert Agricultural Production ---
                    production_id = agri_production_id_counter
                    cursor.execute(
                        """
                        INSERT INTO public."Agricultural_Production"
                        ("ID", year, production_tons, "Product_ID", "Country_ID", number_of_animals, harvested_area_ha)
                        VALUES (%s, %s, %s, %s, %s, %s, %s);
                        """,
                        (production_id, prod_year, prod_tons, product_id, country_id, prod_n_animals, prod_harvested_area)
                    )
                    agri_production_id_counter += 1
                    processed_rows += 1

                except (IndexError, ValueError, TypeError) as e:
                    print(f"Skipping row due to data error: {row} - Error: {e}")
                    skipped_rows += 1
                except psycopg2.Error as e:
                    print(f"Database error processing row: {row} - Error: {e}")
                    conn.rollback()
                    skipped_rows += 1

                # Print progress occasionally
                if (processed_rows + skipped_rows) % 1000 == 0:
                    print(f"  Processed {processed_rows + skipped_rows} rows ({processed_rows} inserted, {skipped_rows} skipped)...", end='\r')
                    sys.stdout.flush() # Force output in Jupyter

    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        raise
    finally:
        print(f"\nFinished loading {file_path}. Total rows processed: {processed_rows + skipped_rows}, Inserted: {processed_rows}, Skipped: {skipped_rows}.")


def load_census_data(cursor, file_path):
    """
    Loads data from the agricultural census CSV into relevant tables.
    Updates Country, Farm_Size_Range, Agricultural_Census, and inserts
    into Rural_Property_Distributuon.

    Args:
        cursor: A psycopg2 cursor object.
        file_path (str): The path to the agricultural_census.csv file.
    """
    print(f"\nLoading data from {file_path}...")
    global country_map, country_id_counter
    global farm_size_range_map, farm_size_range_id_counter
    global agri_census_map, agri_census_id_counter
    global rural_distribution_id_counter

    processed_rows = 0
    skipped_rows = 0
    try:
        with open(file_path, 'r', encoding='utf-8') as csvfile:
            reader = csv.reader(csvfile)
            header = next(reader) # Skip header row

            for row in reader:
                try:
                    # Extract data from CSV row
                    iso_code = row[0].strip()
                    country_name = row[1].strip()
                    census_decade = row[2].strip()
                    census_year = safe_int(row[3])
                    total_area = safe_float(row[4])
                    num_properties = safe_int(row[5])
                    farm_lower = safe_int(row[6])
                    farm_upper = safe_int(row[7]) # Needs careful handling if it represents '> value'

                    # --- Validate essential data ---
                    # Assuming upper limit is required for a valid range entry
                    if not all([iso_code, country_name, census_decade, census_year is not None,
                                total_area is not None, num_properties is not None,
                                farm_lower is not None, farm_upper is not None]):
                        # print(f"Skipping row due to missing essential data: {row}")
                        skipped_rows += 1
                        continue

                    # --- Get or Create Country ID ---
                    # (Reuses logic and map)
                    country_key = (country_name, iso_code)
                    if country_key not in country_map:
                        current_country_id = country_id_counter
                        cursor.execute(
                            """
                            INSERT INTO public."Country" ("ID", name, iso_code)
                            VALUES (%s, %s, %s)
                            ON CONFLICT (name, iso_code) DO NOTHING;
                            """,
                            (current_country_id, country_name, iso_code)
                        )
                        cursor.execute(
                            'SELECT "ID" FROM public."Country" WHERE name = %s AND iso_code = %s',
                            (country_name, iso_code)
                        )
                        result = cursor.fetchone()
                        if result:
                           country_id = result[0]
                           country_map[country_key] = country_id
                           if country_id >= country_id_counter:
                               country_id_counter = country_id + 1
                        else:
                             print(f"Warning: Could not retrieve ID for country: {country_name}, {iso_code}")
                             skipped_rows += 1
                             continue
                    else:
                        country_id = country_map[country_key]

                    # --- Get or Create Farm Size Range ID ---
                    # Handle potential variations in upper limit representation if needed
                    size_range_key = (farm_lower, farm_upper)
                    if size_range_key not in farm_size_range_map:
                        current_range_id = farm_size_range_id_counter
                        cursor.execute(
                            """
                            INSERT INTO public."Farm_Size_Range" ("ID", lower_limit, upper_limit)
                            VALUES (%s, %s, %s)
                            ON CONFLICT (lower_limit, upper_limit) DO NOTHING;
                            """,
                            (current_range_id, farm_lower, farm_upper)
                        )
                        cursor.execute(
                            'SELECT "ID" FROM public."Farm_Size_Range" WHERE lower_limit = %s AND upper_limit = %s',
                            (farm_lower, farm_upper)
                        )
                        result = cursor.fetchone()
                        if result:
                           size_range_id = result[0]
                           farm_size_range_map[size_range_key] = size_range_id
                           if size_range_id >= farm_size_range_id_counter:
                               farm_size_range_id_counter = size_range_id + 1
                        else:
                            print(f"Warning: Could not retrieve ID for farm size range: {farm_lower}-{farm_upper}")
                            skipped_rows += 1
                            continue
                    else:
                        size_range_id = farm_size_range_map[size_range_key]

                    # --- Get or Create Agricultural Census ID ---
                    # A census is unique per country and year
                    census_key = (country_id, census_year)
                    if census_key not in agri_census_map:
                        current_census_id = agri_census_id_counter
                        cursor.execute(
                            """
                            INSERT INTO public."Agricultural_Census" ("ID", census_year, decade, "Country_ID")
                            VALUES (%s, %s, %s, %s)
                            ON CONFLICT ("Country_ID", census_year) DO NOTHING;
                            """,
                            (current_census_id, census_year, census_decade, country_id)
                        )
                        cursor.execute(
                            'SELECT "ID" FROM public."Agricultural_Census" WHERE "Country_ID" = %s AND census_year = %s',
                            (country_id, census_year)
                        )
                        result = cursor.fetchone()
                        if result:
                           census_id = result[0]
                           agri_census_map[census_key] = census_id
                           if census_id >= agri_census_id_counter:
                               agri_census_id_counter = census_id + 1
                        else:
                            print(f"Warning: Could not retrieve ID for census: Country {country_id}, Year {census_year}")
                            skipped_rows += 1
                            continue
                    else:
                        census_id = agri_census_map[census_key]

                    # --- Insert Rural Property Distribution ---
                    # Typo "Distributuon" matches schema
                    distribution_id = rural_distribution_id_counter
                    cursor.execute(
                        """
                        INSERT INTO public."Rural_Property_Distributuon"
                        ("ID", total_area_ha, number_of_properties, "Size_Range_ID", "Agricultural_Census_ID")
                        VALUES (%s, %s, %s, %s, %s);
                        """,
                        (distribution_id, total_area, num_properties, size_range_id, census_id)
                    )
                    rural_distribution_id_counter += 1
                    processed_rows += 1

                except (IndexError, ValueError, TypeError) as e:
                    print(f"Skipping row due to data error: {row} - Error: {e}")
                    skipped_rows += 1
                except psycopg2.Error as e:
                    print(f"Database error processing row: {row} - Error: {e}")
                    conn.rollback()
                    skipped_rows += 1

                # Print progress occasionally
                if (processed_rows + skipped_rows) % 1000 == 0:
                    print(f"  Processed {processed_rows + skipped_rows} rows ({processed_rows} inserted, {skipped_rows} skipped)...", end='\r')
                    sys.stdout.flush() # Force output in Jupyter

    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        raise
    finally:
        print(f"\nFinished loading {file_path}. Total rows processed: {processed_rows + skipped_rows}, Inserted: {processed_rows}, Skipped: {skipped_rows}.")

# 5. Main Execution Block

In [9]:
conn = None # Initialize connection variable
try:
    # Establish database connection
    print("Connecting to the PostgreSQL database...")
    conn = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASS,
        host=DB_HOST,
        port=DB_PORT
    )
    conn.autocommit = False # Ensure operations run in a transaction block by default
    cursor = conn.cursor()
    print("Connection successful.")

    # Create tables (idempotent due to IF NOT EXISTS)
    create_tables(cursor) # This commits the schema changes itself

    # Load data from CSV files
    # It's often better to load lookup tables first if possible, but the
    # current approach handles lookup creation on the fly.
    # Order matters if foreign key constraints were immediate, but they are deferred or checked at commit.
    # The use of ON CONFLICT makes the order less critical for lookup tables.

    # Load Food Security Data
    load_food_security_data(cursor, FOOD_SECURITY_CSV)

    # Load Production Data
    load_production_data(cursor, PRODUCTION_CSV)

    # Load Agricultural Census Data
    load_census_data(cursor, CENSUS_CSV)

    # Commit the transaction for all data loaded
    print("\nCommitting all data changes...")
    conn.commit()
    print("Data loaded and committed successfully!")

except psycopg2.OperationalError as e:
    print(f"\nDatabase connection error: {e}")
    print("Please check if the database server is running and if the connection details (dbname, user, password, host, port) are correct.")
except FileNotFoundError as e:
    print(f"\nError: A specified CSV file was not found.")
    print(f"Missing file: {e.filename}")
    print("Please ensure the CSV files exist at the specified paths.")
    if conn:
        conn.rollback() # Rollback any partial changes if a file is missing mid-process
except psycopg2.Error as e:
    print(f"\nA PostgreSQL error occurred: {e}")
    print(f"SQLSTATE: {e.pgcode}")
    print(f"Error Details: {e.pgerror}")
    if conn:
        print("Rolling back transaction...")
        conn.rollback()
except Exception as e:
    print(f"\nAn unexpected error occurred: {e}")
    if conn:
        print("Rolling back transaction...")
        conn.rollback()
finally:
    # Close the cursor and connection
    if 'cursor' in locals() and cursor:
        cursor.close()
        print("Database cursor closed.")
    if conn:
        conn.close()
        print("Database connection closed.")

Connecting to the PostgreSQL database...
Connection successful.
Creating tables...
Tables created successfully (or already exist).

Loading data from suite_of_food_security_indicators.csv...
  Processed 139000 rows (139000 inserted, 0 skipped)...
Finished loading suite_of_food_security_indicators.csv. Total rows processed: 139538, Inserted: 139538, Skipped: 0.

Loading data from production_crops_livestock.csv...
  Processed 1558000 rows (1554694 inserted, 3306 skipped)...
Finished loading production_crops_livestock.csv. Total rows processed: 1558100, Inserted: 1554794, Skipped: 3306.

Loading data from agricultural_census.csv...
  Processed 2000 rows (2000 inserted, 0 skipped)...
Finished loading agricultural_census.csv. Total rows processed: 2278, Inserted: 2278, Skipped: 0.

Committing all data changes...
Data loaded and committed successfully!
Database cursor closed.
Database connection closed.


# 6. Function to execute queries

In [6]:
def execute_query(query_num, description, sql_query):
    """
    Connects to the database, executes a given SQL query,
    and prints the results using pandas for better formatting.

    Args:
        query_num (int): The number of the query for identification.
        description (str): A brief description of what the query does.
        sql_query (str): The SQL query string to execute.
    """
    conn = None
    print(f"\n--- Query {query_num}: {description} ---")
    print("SQL:")
    print(sql_query)
    print("\nResults:")

    try:
        conn = psycopg2.connect(
            dbname=DB_NAME, user=DB_USER, password=DB_PASS, host=DB_HOST, port=DB_PORT
        )
        cursor = conn.cursor()

        cursor.execute(sql_query)

        if cursor.description: # Check if the query returned columns
            colnames = [desc[0] for desc in cursor.description]
            results = cursor.fetchall()

            if results:
                # Use pandas DataFrame for pretty printing
                df = pd.DataFrame(results, columns=colnames)
                print(df.to_string(index=False)) # Print without index
                df.to_csv(f'query_{query_num}_result.csv', index=False)
            else:
                print("Query executed successfully, but no rows returned.")
        else:
             print("Query executed successfully (e.g., an UPDATE or DELETE with no RETURNING clause).")


    except psycopg2.Error as e:
        print(f"\nDatabase error executing Query {query_num}: {e}")
        print(f"SQLSTATE: {e.pgcode}")
        print(f"Error Details: {e.pgerror}")
        if conn:
            conn.rollback() # Rollback if any error occurs during fetch/execute
    except Exception as e:
        print(f"\nAn unexpected error occurred during Query {query_num}: {e}")
    finally:
        if 'cursor' in locals() and cursor:
            cursor.close()
        if conn:
            conn.close()
    print("-" * (len(description) + 14)) # Separator

# 7. Query 1: Land Concentration vs. Undernourishment (Most Recent Census/Indicator Year).



Description: Calculates the percentage of agricultural land held by large farms (>100 ha)
for the latest available census year per country and compares it with the
'Prevalence of undernourishment' indicator for the closest available year.

This attempts to see if higher land concentration correlates with higher undernourishment.

In [7]:
query1_sql = """
WITH LatestCensus AS (
    -- Find the most recent census year for each country
    SELECT
        "Country_ID",
        MAX(census_year) AS latest_census_year
    FROM public."Agricultural_Census"
    GROUP BY "Country_ID"
),
LandDistribution AS (
    -- Calculate total area and area in large farms (>100ha) for the latest census
    SELECT
        ac."Country_ID",
        ac.census_year,
        SUM(rpd.total_area_ha) AS total_area_latest_census,
        SUM(CASE WHEN fsr.lower_limit >= 100 THEN rpd.total_area_ha ELSE 0 END) AS large_farm_area_latest_census
    FROM public."Rural_Property_Distributuon" rpd -- Typo matches schema
    JOIN public."Farm_Size_Range" fsr ON rpd."Size_Range_ID" = fsr."ID"
    JOIN public."Agricultural_Census" ac ON rpd."Agricultural_Census_ID" = ac."ID"
    JOIN LatestCensus lc ON ac."Country_ID" = lc."Country_ID" AND ac.census_year = lc.latest_census_year
    GROUP BY ac."Country_ID", ac.census_year
),
LatestUndernourishment AS (
    -- Find the most recent undernourishment percentage for each country
    SELECT
        fsm."Country_ID",
        MAX(fsm.year) AS latest_indicator_year
    FROM public."Food_Security_Measurement" fsm
    JOIN public."Food_Security_Indicator" fsi ON fsm."Indicator_ID" = fsi."ID"
    WHERE fsi.name = 'Prevalence of undernourishment (percent) (3-year average)'
    GROUP BY fsm."Country_ID"
),
UndernourishmentValue AS (
    -- Get the actual undernourishment value for the latest year found
    SELECT
        fsm."Country_ID",
        fsm.year,
        fsm.value AS undernourishment_percent
    FROM public."Food_Security_Measurement" fsm
    JOIN LatestUndernourishment lu ON fsm."Country_ID" = lu."Country_ID" AND fsm.year = lu.latest_indicator_year
    JOIN public."Food_Security_Indicator" fsi ON fsm."Indicator_ID" = fsi."ID"
    WHERE fsi.name = 'Prevalence of undernourishment (percent) (3-year average)'
)
-- Final Selection: Join land concentration with undernourishment
SELECT
    c.name AS country_name,
    ld.census_year,
    uv.year AS indicator_year,
    ROUND((ld.large_farm_area_latest_census / ld.total_area_latest_census * 100)::numeric, 2) AS large_farm_area_percent,
    uv.undernourishment_percent
FROM LandDistribution ld
JOIN UndernourishmentValue uv ON ld."Country_ID" = uv."Country_ID"
JOIN public."Country" c ON ld."Country_ID" = c."ID"
WHERE ld.total_area_latest_census > 0 -- Avoid division by zero
ORDER BY large_farm_area_percent DESC NULLS LAST -- Show countries with highest concentration first
LIMIT 50; -- Limit results for brevity"""

execute_query(1, "Land Concentration vs. Undernourishment", query1_sql)


--- Query 1: Land Concentration vs. Undernourishment ---
SQL:

WITH LatestCensus AS (
    -- Find the most recent census year for each country
    SELECT
        "Country_ID",
        MAX(census_year) AS latest_census_year
    FROM public."Agricultural_Census"
    GROUP BY "Country_ID"
),
LandDistribution AS (
    -- Calculate total area and area in large farms (>100ha) for the latest census
    SELECT
        ac."Country_ID",
        ac.census_year,
        SUM(rpd.total_area_ha) AS total_area_latest_census,
        SUM(CASE WHEN fsr.lower_limit >= 100 THEN rpd.total_area_ha ELSE 0 END) AS large_farm_area_latest_census
    FROM public."Rural_Property_Distributuon" rpd -- Typo matches schema
    JOIN public."Farm_Size_Range" fsr ON rpd."Size_Range_ID" = fsr."ID"
    JOIN public."Agricultural_Census" ac ON rpd."Agricultural_Census_ID" = ac."ID"
    JOIN LatestCensus lc ON ac."Country_ID" = lc."Country_ID" AND ac.census_year = lc.latest_census_year
    GROUP BY ac."Country_ID", ac.censu

# 8. Query 2: Staple Crop Production (Rice, Wheat, Maize) vs. Dietary Energy Adequacy


Description: Compares the total production (in tons) of major staple cereals (Rice, Wheat, Maize)
for a specific year (e.g., 2018) with the 'Average dietary energy supply adequacy'
indicator for the same year in various countries.

This explores if higher staple production correlates with better energy supply adequacy.

In [8]:
query2_sql = """
WITH StapleProduction AS (
    -- Sum production of Rice, Wheat, and Maize for year 2018 per country
    SELECT
        ap."Country_ID",
        SUM(ap.production_tons) AS total_staple_tons_2018
    FROM public."Agricultural_Production" ap
    JOIN public."Agricultural_Product" apr ON ap."Product_ID" = apr."ID"
    WHERE ap.year = 2018
      AND apr.name IN ('Rice', 'Wheat', 'Maize (corn)') -- Key staple crops
    GROUP BY ap."Country_ID"
),
EnergyAdequacy AS (
    -- Get the Dietary Energy Supply Adequacy for 2018 per country
    SELECT
        fsm."Country_ID",
        fsm.value AS energy_adequacy_percent_2018
    FROM public."Food_Security_Measurement" fsm
    JOIN public."Food_Security_Indicator" fsi ON fsm."Indicator_ID" = fsi."ID"
    WHERE fsm.year = 2018
      AND fsi.name = 'Average dietary energy supply adequacy (percent) (3-year average)'
)
-- Final Selection: Join production with the indicator
SELECT
    c.name AS country_name,
    sp.total_staple_tons_2018,
    ea.energy_adequacy_percent_2018
FROM StapleProduction sp
JOIN EnergyAdequacy ea ON sp."Country_ID" = ea."Country_ID"
JOIN public."Country" c ON sp."Country_ID" = c."ID"
ORDER BY ea.energy_adequacy_percent_2018 DESC NULLS LAST -- Show countries with highest adequacy first
LIMIT 50; -- Limit results
"""


execute_query(2, "Staple Crop Production vs. Dietary Energy Adequacy (2018)", query2_sql)


--- Query 2: Staple Crop Production vs. Dietary Energy Adequacy (2018) ---
SQL:

WITH StapleProduction AS (
    -- Sum production of Rice, Wheat, and Maize for year 2018 per country
    SELECT
        ap."Country_ID",
        SUM(ap.production_tons) AS total_staple_tons_2018
    FROM public."Agricultural_Production" ap
    JOIN public."Agricultural_Product" apr ON ap."Product_ID" = apr."ID"
    WHERE ap.year = 2018
      AND apr.name IN ('Rice', 'Wheat', 'Maize (corn)') -- Key staple crops
    GROUP BY ap."Country_ID"
),
EnergyAdequacy AS (
    -- Get the Dietary Energy Supply Adequacy for 2018 per country
    SELECT
        fsm."Country_ID",
        fsm.value AS energy_adequacy_percent_2018
    FROM public."Food_Security_Measurement" fsm
    JOIN public."Food_Security_Indicator" fsi ON fsm."Indicator_ID" = fsi."ID"
    WHERE fsm.year = 2018
      AND fsi.name = 'Average dietary energy supply adequacy (percent) (3-year average)'
)
-- Final Selection: Join production with the indicator

# 9. Query 3: Animal Protein Supply vs. Meat/Milk/Egg Production


Description: Examines the relationship between the 'Average supply of protein of animal origin'
and the combined production volume (tons) of major animal products ('Meat, Total',
'Milk, Total', 'Eggs Primary') for the most recent year available for both datasets.

This checks if higher production aligns with higher per capita animal protein supply.

In [9]:
query3_sql = """
WITH LatestYear AS (
    -- Find the latest year present in both Production and Food Security datasets for key items
    SELECT MAX(T.yr) as latest_common_year
    FROM (
        SELECT year as yr FROM public."Agricultural_Production" ap
        JOIN public."Agricultural_Product" apr ON ap."Product_ID" = apr."ID"
        WHERE apr.name IN ('Meat, Total', 'Milk, Total', 'Eggs Primary')
        INTERSECT
        SELECT year as yr FROM public."Food_Security_Measurement" fsm
        JOIN public."Food_Security_Indicator" fsi ON fsm."Indicator_ID" = fsi."ID"
        WHERE fsi.name = 'Average supply of protein of animal origin (g/cap/day) (3-year average)'
    ) AS T
),
AnimalProduction AS (
    -- Sum production of Meat, Milk, Eggs for the latest common year
    SELECT
        ap."Country_ID",
        SUM(ap.production_tons) AS total_animal_product_tons
    FROM public."Agricultural_Production" ap
    JOIN public."Agricultural_Product" apr ON ap."Product_ID" = apr."ID"
    JOIN LatestYear ly ON ap.year = ly.latest_common_year
    WHERE apr.name IN ('Meat, Total', 'Milk, Total', 'Eggs Primary')
    GROUP BY ap."Country_ID"
),
AnimalProteinSupply AS (
    -- Get Animal Protein Supply for the latest common year
    SELECT
        fsm."Country_ID",
        fsm.value AS animal_protein_supply_g_cap_day
    FROM public."Food_Security_Measurement" fsm
    JOIN public."Food_Security_Indicator" fsi ON fsm."Indicator_ID" = fsi."ID"
    JOIN LatestYear ly ON fsm.year = ly.latest_common_year
    WHERE fsi.name = 'Average supply of protein of animal origin (g/cap/day) (3-year average)'
)
-- Final Selection: Join production and supply data
SELECT
    c.name AS country_name,
    (SELECT latest_common_year FROM LatestYear) AS data_year,
    apr.total_animal_product_tons,
    aps.animal_protein_supply_g_cap_day
FROM AnimalProduction apr
JOIN AnimalProteinSupply aps ON apr."Country_ID" = aps."Country_ID"
JOIN public."Country" c ON apr."Country_ID" = c."ID"
WHERE apr.total_animal_product_tons IS NOT NULL AND aps.animal_protein_supply_g_cap_day IS NOT NULL
ORDER BY aps.animal_protein_supply_g_cap_day DESC -- Show countries with highest supply first
LIMIT 50;
"""

execute_query(3, "Animal Protein Supply vs. Meat/Milk/Egg Production (Latest Year)", query3_sql)


--- Query 3: Animal Protein Supply vs. Meat/Milk/Egg Production (Latest Year) ---
SQL:

WITH LatestYear AS (
    -- Find the latest year present in both Production and Food Security datasets for key items
    SELECT MAX(T.yr) as latest_common_year
    FROM (
        SELECT year as yr FROM public."Agricultural_Production" ap
        JOIN public."Agricultural_Product" apr ON ap."Product_ID" = apr."ID"
        WHERE apr.name IN ('Meat, Total', 'Milk, Total', 'Eggs Primary')
        INTERSECT
        SELECT year as yr FROM public."Food_Security_Measurement" fsm
        JOIN public."Food_Security_Indicator" fsi ON fsm."Indicator_ID" = fsi."ID"
        WHERE fsi.name = 'Average supply of protein of animal origin (g/cap/day) (3-year average)'
    ) AS T
),
AnimalProduction AS (
    -- Sum production of Meat, Milk, Eggs for the latest common year
    SELECT
        ap."Country_ID",
        SUM(ap.production_tons) AS total_animal_product_tons
    FROM public."Agricultural_Production" ap
    JO

# 10. Query 4: Change Over Time - Soya Bean Production and Protein Supply in Brazil


Description: Tracks the annual production of 'Soya beans' alongside the 'Average protein supply'
indicator in Brazil over the available years to see how they trend together.

In [10]:
query4_sql = """
SELECT
    c.name AS country_name,
    ap.year,
    ap.production_tons AS soya_beans_tons,
    fsm.value AS avg_protein_supply_g_cap_day
FROM public."Agricultural_Production" ap
JOIN public."Agricultural_Product" apr ON ap."Product_ID" = apr."ID"
JOIN public."Country" c ON ap."Country_ID" = c."ID"
LEFT JOIN public."Food_Security_Measurement" fsm ON ap."Country_ID" = fsm."Country_ID" AND ap.year = fsm.year
LEFT JOIN public."Food_Security_Indicator" fsi ON fsm."Indicator_ID" = fsi."ID"
    AND fsi.name = 'Average protein supply (g/cap/day) (3-year average)' -- Protein indicator
WHERE
    c.name = 'Brazil' -- Specific country
    AND apr.name = 'Soya beans' -- Specific product
ORDER BY ap.year;
"""

execute_query(4, "Trend: Soya Bean Production vs. Protein Supply in Brazil", query4_sql)


--- Query 4: Trend: Soya Bean Production vs. Protein Supply in Brazil ---
SQL:

SELECT
    c.name AS country_name,
    ap.year,
    ap.production_tons AS soya_beans_tons,
    fsm.value AS avg_protein_supply_g_cap_day
FROM public."Agricultural_Production" ap
JOIN public."Agricultural_Product" apr ON ap."Product_ID" = apr."ID"
JOIN public."Country" c ON ap."Country_ID" = c."ID"
LEFT JOIN public."Food_Security_Measurement" fsm ON ap."Country_ID" = fsm."Country_ID" AND ap.year = fsm.year
LEFT JOIN public."Food_Security_Indicator" fsi ON fsm."Indicator_ID" = fsi."ID"
    AND fsi.name = 'Average protein supply (g/cap/day) (3-year average)' -- Protein indicator
WHERE
    c.name = 'Brazil' -- Specific country
    AND apr.name = 'Soya beans' -- Specific product
ORDER BY ap.year;


Results:
country_name  year  soya_beans_tons avg_protein_supply_g_cap_day
      Brazil  1961         271488.0                         None
      Brazil  1962         345175.0                         None
      Brazil

# 11. Query 5: Detailed Land Distribution Profile for a Country (e.g., Brazil, 2017 Census)


Description: Shows the detailed breakdown of the number of properties and total area
for each farm size range for a specific country and census year (e.g., Brazil 2017).

Also calculates the percentage of total properties and total area in each size bracket.

In [12]:
query5_sql = """
WITH CensusData AS (
    -- Get distribution data for Brazil 2017 census
    SELECT
        rpd."Size_Range_ID",
        fsr.lower_limit,
        fsr.upper_limit,
        rpd.number_of_properties,
        rpd.total_area_ha
    FROM public."Rural_Property_Distributuon" rpd -- Typo matches schema
    JOIN public."Agricultural_Census" ac ON rpd."Agricultural_Census_ID" = ac."ID"
    JOIN public."Country" c ON ac."Country_ID" = c."ID"
    JOIN public."Farm_Size_Range" fsr ON rpd."Size_Range_ID" = fsr."ID"
    WHERE c.name = 'Brazil' AND ac.census_year = 2017
),
Totals AS (
    -- Calculate total properties and area for this census
    SELECT
        SUM(number_of_properties) AS grand_total_properties,
        SUM(total_area_ha) AS grand_total_area_ha
    FROM CensusData
)
-- Final Selection: Show details per size range with percentages
SELECT
    cd.lower_limit || ' - ' || cd.upper_limit || ' ha' AS farm_size_range,
    cd.number_of_properties,
    ROUND((cd.number_of_properties::numeric / t.grand_total_properties * 100), 2) AS percent_of_total_properties,
    ROUND(cd.total_area_ha::numeric, 1) AS total_area_in_range_ha,
    ROUND((cd.total_area_ha::numeric / t.grand_total_area_ha * 100)::numeric, 2) AS percent_of_total_area
FROM CensusData cd, Totals t -- Cross join with totals (only 1 row in Totals)
ORDER BY cd.lower_limit;
"""


execute_query(5, "Detailed Land Distribution Profile (Brazil, 2017)", query5_sql)


--- Query 5: Detailed Land Distribution Profile (Brazil, 2017) ---
SQL:

WITH CensusData AS (
    -- Get distribution data for Brazil 2017 census
    SELECT
        rpd."Size_Range_ID",
        fsr.lower_limit,
        fsr.upper_limit,
        rpd.number_of_properties,
        rpd.total_area_ha
    FROM public."Rural_Property_Distributuon" rpd -- Typo matches schema
    JOIN public."Agricultural_Census" ac ON rpd."Agricultural_Census_ID" = ac."ID"
    JOIN public."Country" c ON ac."Country_ID" = c."ID"
    JOIN public."Farm_Size_Range" fsr ON rpd."Size_Range_ID" = fsr."ID"
    WHERE c.name = 'Brazil' AND ac.census_year = 2017
),
Totals AS (
    -- Calculate total properties and area for this census
    SELECT
        SUM(number_of_properties) AS grand_total_properties,
        SUM(total_area_ha) AS grand_total_area_ha
    FROM CensusData
)
-- Final Selection: Show details per size range with percentages
SELECT
    cd.lower_limit || ' - ' || cd.upper_limit || ' ha' AS farm_size_range,
